In [49]:
import fourier_hardware_py
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr2t2v2")
# 配置文件中有灵巧手相关字段 
config.getHandNames()


['left_hand', 'right_hand']

In [50]:
left_hand_type = config.getHandType("left_hand")
# 灵巧手类型，有FDH-6L,FDH-6R,FDH-12L,FDH-12R
left_hand_type

'FDH-6L'

In [51]:
right_hand_type = config.getHandType("right_hand")
right_hand_type

'FDH-6R'

In [52]:
# 根据灵巧手的名字获得灵巧手的ip
left_hand_ip = config.getHandIP("left_hand")
left_hand_ip

'192.168.137.19'

In [53]:
right_hand_ip = config.getHandIP("right_hand")
right_hand_ip

'192.168.137.39'

In [ ]:
# 构造灵巧手的控制对象
dexhand = fourier_hardware_py.DexHand()

In [ ]:
# 灵巧手init()，注意可以根据返回值判断是否init成功
ret = dexhand.init()
ret

In [ ]:
if ret == fourier_hardware_py.DexRet.FAIL:
    print("dexhand init failed")
elif ret == fourier_hardware_py.DexRet.SUCCESS:
    print("dexhand init success")

In [ ]:
# 获取灵巧手init时检测到的ip，可以配置文件中的ip比较，确定ip是否正确
ip_list = dexhand.get_ip_list()
ip_list

In [ ]:
# position (in radians), velocity, and current values for finger joints. 

# FDH12 Default Mode: Position control (radians) using P parameter. In PD control mode, uses PVC parameters (Position/Velocity feedforward/Current feedforward).
# FDH12 Joint Range Limits (Radians):
# Index1-3：0.2～1.69, 0.01～1.43, -0.04～0.26
# Middle4-5：0.2～1.69, 0.01～1.43
# Ring6-7：0.2～1.69, 0.01～1.43
# Little8-9：0.2～1.69, 0.01～1.43
# Thumb10-12：-0.02～1.23, 0.14～1.35, 0.2～1.57

# FDH6 Default Mode: Position-Velocity control (PV parameters).
# FDH6 Joint Range Limits (Radians):
# Index1：0.17～1.78
# Middle2：0.17～1.78
# Ring3：0.17～1.78
# Little4：0.17～1.78
# Thumb5-6：0.12-1.28, 0.0-1.68

print(dexhand.get_pvc(left_hand_ip))
print(dexhand.get_pvc(right_hand_ip))

command_list = [[0]*6, [0]*6, [0]*6]
dexhand.set_pvc(left_hand_ip, command_list)
dexhand.set_pvc(right_hand_ip, command_list)

In [ ]:
# 获取灵巧手位置，返回一个数组，空代表获取失败，最大等待时间100ms 
# 50 hz 频率get pos，1000次中平均有10次超时，超时时间平均为0.022s，没有发现大规模超时，平均单手get时间约为0.01s，建议一手一线程，get和set不要再一个线程。
# set线程很快，不会造成线程堵塞，但是会受到get函数的影响。
import threading
import time
import math

# 生成一个放缩到规定范围内的sin函数
def rescale_sinus (phase, min_value, max_value):
    y = (max_value - min_value)/2*math.sin(phase) + (max_value + min_value)/2
    return y


def hand_loop(name, ip):
    count = 0
    start_time = time.perf_counter()
    phase = -1.57

    while True:
        loop_start = time.perf_counter()

        # 控制值可变化（模拟控制）
        phase += 0.02
      
        # 以pvc(position, velocity, current)的形式生成控制指令，position是一个正弦曲线，velocity和current为0
        command_p = [rescale_sinus(phase,0.17,1.78), # Index1
            rescale_sinus(phase,0.17,1.78),          # Middle2
            rescale_sinus(phase,0.17,1.78),          # Ring3
            rescale_sinus(phase,0.17,1.78),          # Little4
            rescale_sinus(phase,0.12,1.28),          # Thumb5
            0,                                       # Thumb6
            ]
        command_v = [0]*6
        command_c = [0]*6
        command_list = [command_p, command_v, command_c]
                
        dexhand.set_pvc(ip, command_list)

        count += 1
        now = time.perf_counter()

        if now - start_time >= 1.0:
            freq = count / (now - start_time)
            # print(f"[{name}] 频率: {freq:.2f} Hz")
            start_time = now
            count = 0

        # 精准睡眠（尝试压到100Hz）
        duration = time.perf_counter() - loop_start
        if duration < 0.01:
            time.sleep(0.01 - duration)

# 启动两个线程
threading.Thread(target=hand_loop, args=("left", left_hand_ip), daemon=True).start()
threading.Thread(target=hand_loop, args=("right", right_hand_ip), daemon=True).start()


In [63]:
# 上电自动enable
# disable之后需要enable
dexhand.enable()

<DexRet.SUCCESS: 0>

In [ ]:
# 去使能
dexhand.disable()

<DexRet.SUCCESS: 0>